## Fragmentación de escritura del índice secundario global
La clave principal de una tabla de DynamoDB o un índice secundario global consta de una clave de partición y una clave de ordenación opcional. La forma de diseñar el contenido de estas claves es extremadamente importante para la estructura y el rendimiento de su base de datos. Los valores de la clave de partición determinan las particiones lógicas en las que se almacenan los datos. Por lo tanto, es importante elegir un valor de clave de partición que distribuya uniformemente la carga de trabajo entre todas las particiones de la tabla o índice secundario global.

En este ejercicio, aprenderá sobre la fragmentación de escritura del índice secundario global, que es un patrón de diseño eficaz para consultar de forma selectiva los elementos distribuidos en diferentes particiones lógicas de una tabla muy grande. Revisemos el ejemplo de los registros de acceso al servidor del Ejercicio 1, que está basado en los registros de acceso al servicio de Apache. Esta vez, consulta los elementos con código de respuesta 4xx. Tenga en cuenta que los elementos con código de respuesta 4xx son un porcentaje muy pequeño de los datos totales y no tienen una distribución uniforme por código de respuesta. Por ejemplo, el código de respuesta 200 OK tiene más registros que los demás, lo cual es esperable para cualquier aplicación web.

El siguiente gráfico muestra la distribución de los registros de log por código de respuesta para el archivo de muestra, logfile_medium1.csv.

![img](images/image8.jpg)

In [105]:
import boto3
from botocore.exceptions import ClientError
import pandas as pd
import numpy as np
import sys
sys.path.append('../')
from spdynamodb import DynamoTable
import random
import json
import time
from decimal import Decimal

In [2]:
dt = DynamoTable()
try:
    dt.select_table('logfile')
    print(dt)
except:
    dt.create_table(
        table_name='logfile',
        partition_key='PK',
        partition_key_type='S',
        read_capacity=5,
        write_capacity=5,
        tags=[{'Key': 'workshop-design-patterns', 'Value': 'targeted-for-cleanup'}]
)

Table info:
 - Table name: logfile
 - Table arn: arn:aws:dynamodb:us-east-1:637423169504:table/logfile
 - Table creation: 2024-07-24T11:04:19
 - Key schema: [{'AttributeName': 'PK', 'KeyType': 'HASH'}]
 - Attribute definitions: [{'AttributeName': 'GSI1-PK', 'AttributeType': 'S'}, {'AttributeName': 'GSI1-SK', 'AttributeType': 'S'}, {'AttributeName': 'PK', 'AttributeType': 'S'}]
 - Point-in-time recovery status: DISABLED
 - Delete protection: False
 - Stream enabled: OFF
 - Tags: [{'Key': 'workshop-design-patterns', 'Value': 'targeted-for-cleanup'}]



Creará un índice secundario global de escritura separada en una tabla para aleatorizar las escrituras a través de múltiples valores de clave de partición lógica. En efecto, esto aumenta el rendimiento de escritura y lectura de la aplicación. Para aplicar este patrón de diseño, puedes crear un número aleatorio de un conjunto fijo (por ejemplo, de 1 a 10), y utilizar este número como la clave de partición lógica para un índice secundario global. Al aleatorizar la clave de partición, las escrituras en la tabla se distribuyen uniformemente entre todos los valores de la clave de partición, independientemente de cualquier atributo. Esto producirá un mejor paralelismo y un mayor rendimiento general. Además, si la aplicación necesita consultar los registros por un código de respuesta específico en una fecha específica, puede crear una clave de ordenación compuesta utilizando una combinación del código de respuesta y la fecha.

En este ejercicio, se crea un índice secundario global utilizando valores aleatorios para la clave de partición y la clave compuesta `responsecode#date#hourofday` como clave de ordenación. La tabla logfile_scan que creó y rellenó durante la fase de preparación del taller ya tiene estos dos atributos. Si no completó los pasos de configuración, vuelva a Configuración - Paso 6 y complete el paso. Estos atributos se crearon utilizando el siguiente código.

In [4]:
dt.create_global_secondary_index(
    att_name="GSI1-PK",
    att_type="S",
    sort_index="GSI1-SK",
    sort_type="S",
    index_name="GSI1",
    proj_type=["bytessent"],
    read_capacity=5,
    write_capacity=5
)

In [5]:
status = dt.check_status_gsi()
while status == 'CREATING':
    status = dt.check_status_gsi()
    time.sleep(30)
print("Global secondary index created.")

In [124]:
columns = ['requestid','host','date','hourofday','timezone','method','url','responsecode','bytessent','useragent']
df_logs = pd.read_csv('data/logfile_small1.csv', names=columns, header=None)

df_logs['PK'] = df_logs['requestid'].astype(str)
df_logs['PK'] = df_logs.apply(lambda x: str("request#" + x['PK']), axis=1)

SHARDS = 10
df_logs['GSI1-PK'] = df_logs.apply(lambda x: 'shard' + '#' + str(random.randint(1, SHARDS)), axis=1)
df_logs['GSI1-SK'] = df_logs.apply(lambda x: str(x['responsecode']) + '#' + x['date'] + '#' + str(x['hourofday']), axis=1)

In [125]:
df_logs.head()

,requestid,host,date,hourofday,timezone,method,url,responsecode,bytessent,useragent,PK,GSI1-PK,GSI1-SK
0,1,66.249.67.3,2009-07-20,20,GMT-0700,GET,/gallery/main.php?g2_controller=exif.SwitchDet...,302,5,Mozilla/5.0 (compatible; Googlebot/2.1; +http:...,request#1,shard#3,302#2009-07-20#20
1,2,66.249.67.3,2009-07-20,20,GMT-0700,GET,/gallery/main.php?g2_itemId=15741&g2_fromNavId...,200,8068,Mozilla/5.0 (compatible; Googlebot/2.1; +http:...,request#2,shard#7,200#2009-07-20#20
2,3,64.233.172.17,2009-07-20,20,GMT-0700,GET,/gwidgets/alexa.xml,200,2969,Mozilla/5.0 (compatible) Feedfetcher-Google; (...,request#3,shard#9,200#2009-07-20#20
3,4,74.125.74.193,2009-07-20,20,GMT-0700,GET,/gwidgets/alexa.xml,200,2969,Mozilla/5.0 (compatible) Feedfetcher-Google; (...,request#4,shard#7,200#2009-07-20#20
4,5,192.168.1.198,2009-07-20,20,GMT-0700,GET,/,200,17935,Mozilla/5.0 (Macintosh; U; Intel Mac OS X 10_5...,request#5,shard#7,200#2009-07-20#20


In [110]:
dt.batch_pandas(df_logs)

Data loaded successfully in 10.55 seconds.


In [126]:
df_logs.responsecode.value_counts()

responsecode
200    420
302     66
404      8
304      6
Name: count, dtype: int64

In [132]:
df = pd.DataFrame()
for i in range(1, SHARDS + 1):
    df_c = dt.query(
        pk_value=f"shard#{i}",
        sk_value="302*",
        index_name="GSI1", 
        to_pandas=True
    )
    df_c = pd.concat([df, df_c])
    df = df_c

In [135]:
df.head()

,GSI1-SK,bytessent,GSI1-PK,PK
0,302#2009-07-20#20,5,shard#1,request#145
1,302#2009-07-20#20,5,shard#1,request#114
2,302#2009-07-20#21,5,shard#1,request#287
3,302#2009-07-20#21,0,shard#1,request#378
4,302#2009-07-20#21,5,shard#1,request#438


In [134]:
len(df)

66